*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split

In [ ]:
# Import data
dataset_path = keras.utils.get_file("insurance.csv", "https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv")
dataset = pd.read_csv(dataset_path)
dataset.tail()


In [ ]:
dataset_copy = dataset.copy()
print(dataset_copy.head())
CATEGORICAL_COLUMNS = ['smoker', 'region', 'sex']
NUMERICAL_COLUMNS = ['age', 'bmi', 'children']
# convert categorical to codes
for cat_col in CATEGORICAL_COLUMNS:
  dataset_copy[cat_col] = pd.Categorical(dataset_copy[cat_col])
  dataset_copy[cat_col] = dataset_copy[cat_col].cat.codes
print('After categorical column conversion')
print(dataset_copy.head())
print('Get Targets')
target = dataset_copy.pop('expenses')
print(dataset_copy.head())
print(target.head())

tf_dataset = tf.data.Dataset.from_tensor_slices((dataset_copy.values, target.values))

DATASET_SIZE = tf.data.experimental.cardinality(tf_dataset).numpy()
BATCH_SIZE = 32 

TRAIN_SIZE = int(0.7 * DATASET_SIZE)
VAL_SIZE = int(0.15 * DATASET_SIZE)
TEST_SIZE = int(0.15 * DATASET_SIZE)

full_dataset = tf_dataset.shuffle(DATASET_SIZE, reshuffle_each_iteration=False)
train_dataset = full_dataset.take(TRAIN_SIZE).repeat().batch(BATCH_SIZE)
test_dataset = full_dataset.skip(TRAIN_SIZE)
val_dataset = test_dataset.skip(VAL_SIZE).repeat().batch(BATCH_SIZE)
test_dataset = test_dataset.take(TEST_SIZE).repeat().batch(BATCH_SIZE)


In [ ]:
dataset_copy = dataset.copy()
train, test = train_test_split(dataset_copy, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

# Define method to create tf.data dataset from Pandas Dataframe
# This worked with tf 2.0 but does not work with tf 2.2
def df_to_dataset_tf_2_0(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    #labels = dataframe.pop(label_column)
    labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

def df_to_dataset(dataframe, label_column, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop(label_column)
    #labels = dataframe[label_column]

    ds = tf.data.Dataset.from_tensor_slices((dataframe.to_dict(orient='list'), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


batch_size = 32 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, label_column = 'expenses', batch_size=batch_size)
val_ds = df_to_dataset(val,label_column = 'expenses',  shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, label_column = 'expenses', shuffle=False, batch_size=batch_size)


In [ ]:
feature_columns = []
feature_layer_inputs = {}

def normalizer(feature):
  def minmax(x):
    mini = dataset_copy[feature].min()
    maxi = dataset_copy[feature].max()
    return (x - mini)/(maxi-mini)
  return(minmax)

for header in CATEGORICAL_COLUMNS:
  vocabulary = dataset_copy[header].unique()
  cat_c = tf.feature_column.categorical_column_with_vocabulary_list(header, vocabulary)
  one_hot = tf.feature_column.indicator_column(cat_c)
  feature_columns.append(one_hot)
  feature_layer_inputs[header] = tf.keras.Input(shape=(1,), name=header, dtype=tf.string)

for header in NUMERICAL_COLUMNS:
  normalizer_fn = normalizer(header)
  feature_columns.append(tf.feature_column.numeric_column(header, normalizer_fn=normalizer_fn))
  feature_layer_inputs[header] = tf.keras.Input(shape=(1,), name=header)


feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
x = layers.Dense(128, activation='relu')(feature_layer_outputs)
x = layers.Dense(64, activation='relu')(x)
pred = layers.Dense(1)(x)

model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=pred)

model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae', 'mse'])

model.fit(train_ds, epochs=1000)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_ds, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_ds).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
